In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random

random.seed()

Scraping web page function

In [ ]:
def get_article(soup, articles):
    name = soup.find('div', class_='post-title').find('h1').text

    post = soup.find('div', class_='post-text-content')
    article = ''
    for p in post.find_all('p', recursive=False):
        t = p.find(string=True, recursive=False)
        if t != None:
            article += t.text + '\n'

    comments = []
    for div in soup.find_all('div', class_='single-comment'):
        comment = div.find('p').text.strip()
        author = div.find('span', class_='comntr-nm').text

        comments.append({
            'comment': comment,
            'author': author,
        })

    articles.append({
        'name': name,
        'content': article,
        'comments': comments
    })
    return

Initialization of crawling variables

In [ ]:
articles = []

base = 'https://show.blitz.bg/'
stack = [base]
visited = set()

Crawling

In [ ]:
while len(stack) > 0:
    time.sleep(random.randint(0, 50)/100) # sleep random milliseconds
    print("-" , end=" ")
    url = stack.pop()
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
    except Exception as e:
        print(e)
        continue

    try: get_article(soup, articles)
    except Exception as e: pass

    for link_element in soup.select("a[href]"):
        url = link_element['href']
        if base in url and url not in visited:
            visited.add(url)
            stack.append(url)

Saving to file

In [ ]:
import json
import codecs

json_object = json.dumps(articles, indent=4, ensure_ascii=False)
with codecs.open("data/blitz_articles_v2.json", "w", "utf-8") as outfile:
    outfile.write(json_object)

In [ ]:
comments = [comment for article in articles for comment in article['comments']]
json_object = json.dumps(comments, indent=4, ensure_ascii=False)
with codecs.open("data/blitz_comments_v2.json", "w", "utf-8") as outfile:
    outfile.write(json_object)